In [1]:
from river import metrics, compose, preprocessing
from river.datasets import Bikes
from river_torch.regression import Regressor
import torch
from itertools import islice
import pandas as pd

In [2]:
def build_fn(n_features):
    net = torch.nn.Sequential(
        torch.nn.Linear(n_features, 3),
        torch.nn.ReLU(),
        torch.nn.Linear(3, 1),
        torch.nn.Sigmoid()
    )
    return net

def batcher(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [5]:
dataset = Bikes()
metric = metrics.MAE()
optimizer_fn = torch.optim.SGD

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= Regressor(build_fn=build_fn, loss_fn="mse", optimizer_fn=optimizer_fn)

for batch in batcher(dataset.take(5000),5):
    x,y = zip(*batch)
    x = pd.DataFrame(x)
    y_pred = model_pipeline.predict_many(X=x)
    #metric.update(y_true=list(y), y_pred=y_pred)
    _ = [metric.update(y_i, y_pred[idx]) for idx, y_i in enumerate(list(y))] # update the meetric
    model_pipeline.learn_many(X=x, y=y)
print(f'MAE: {metric.get()}')

MAE: 6.927137108176953
